In [ ]:
!pip3 install ktrain

     |████████████████████████████████| 25.3MB 1.4MB/s 
     |████████████████████████████████| 6.8MB 63.0MB/s 
     |████████████████████████████████| 983kB 45.5MB/s 
     |████████████████████████████████| 266kB 46.4MB/s 
     |████████████████████████████████| 1.3MB 48.7MB/s 
     |████████████████████████████████| 1.2MB 43.6MB/s 
     |████████████████████████████████| 471kB 34.6MB/s 
     |████████████████████████████████| 890kB 35.3MB/s 
     |████████████████████████████████| 2.9MB 31.2MB/s 
  Created wheel for ktrain: filename=ktrain-0.25.4-cp36-none-any.whl size=25276443 sha256=5c4e52778b91e6213cede1dc803a6b95014a36c855e8d67cb4caabf0d4901638
  Stored in directory: /root/.cache/pip/wheels/1b/77/8a/bdceaabc308e7178d575278bf6143b7d1a9b939a1e40c56b88
  Created wheel for langdetect: filename=langdetect-1.0.8-cp36-none-any.whl size=993194 sha256=fe10d8e4e42e632f57f1d41676274f57404468a8a3ec374be50d202e65c526aa
  Stored in directory: /root/.cache/pip/wheels/8d/b3/aa/6d99de9f3841d7d3d4

In [ ]:
import pandas as pd
import numpy as np

import ktrain
from ktrain import text

In [ ]:
cd drive/MyDrive/bert-project-data/

/content/drive/MyDrive/bert-project-data


In [ ]:
data_train = pd.read_csv('./data_train.csv', encoding='utf-8')
data_test = pd.read_csv('./data_test.csv', encoding='utf-8')


X_train = data_train.Text.tolist()
X_test = data_test.Text.tolist()

y_train = data_train.Emotion.tolist()
y_test = data_test.Emotion.tolist()

data = data_train.append(data_test, ignore_index=True)

class_names = ['joy', 'sadness', 'fear', 'anger', 'neutral']

print('size of training set: %s' % (len(data_train['Text'])))
print('size of validation set: %s' % (len(data_test['Text'])))
print(data.Emotion.value_counts())

data.head(10)

SIZE_TRAIN = 1000
X_train = X_train[:1000]
y_train = y_train[:1000]

size of training set: 7934
size of validation set: 3393
joy        2326
sadness    2317
anger      2259
neutral    2254
fear       2171
Name: Emotion, dtype: int64


In [ ]:
encoding = {
    'joy': 0,
    'sadness': 1,
    'fear': 2,
    'anger': 3,
    'neutral': 4
}

# Integer values for each class


# encoding_2_modified = {value:key for key, value in encoding.items()}
y_train_2 = [encoding[x] for x in y_train]
y_test = [encoding[x] for x in y_test]
# encoding_2_modified[4]

In [ ]:
(x_train,  y_train_2), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train_2,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=class_names,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


task: text classification


In [ ]:
model = text.text_classifier('bert', train_data=(x_train, y_train_2), preproc=preproc)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train_2), 
                             val_data=(x_test, y_test),
                             batch_size=6)

In [ ]:
learner.fit_onecycle(2e-5, 3)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/3
167/167 [==============================] - 244s 1s/step - loss: 1.5820 - accuracy: 0.3152 - val_loss: 1.1753 - val_accuracy: 0.5423
Epoch 2/3
167/167 [==============================] - 226s 1s/step - loss: 0.9608 - accuracy: 0.6738 - val_loss: 0.7635 - val_accuracy: 0.7235
Epoch 3/3
167/167 [==============================] - 225s 1s/step - loss: 0.3719 - accuracy: 0.8984 - val_loss: 0.7333 - val_accuracy: 0.7321


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=class_names)

              precision    recall  f1-score   support

         joy       0.77      0.79      0.78       707
     sadness       0.66      0.75      0.70       676
        fear       0.85      0.63      0.73       679
       anger       0.70      0.70      0.70       693
     neutral       0.72      0.78      0.75       638

    accuracy                           0.73      3393
   macro avg       0.74      0.73      0.73      3393
weighted avg       0.74      0.73      0.73      3393



array([[560,  37,  11,  25,  74],
       [ 28, 507,  23,  81,  37],
       [ 33, 107, 431,  86,  22],
       [ 36,  77,  34, 486,  60],
       [ 72,  38,   9,  19, 500]])

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.get_classes()

['joy', 'sadness', 'fear', 'anger', 'neutral']

In [ ]:
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: sadness (0.10)


In [ ]:
# let's save the predictor for later use
predictor.save("models/bert_model")

In [ ]:
#testing if predictor can loaded from storage
#also run install ktrain
#import ktrain syntaxes when you solely loading model from saved storage
predictor = ktrain.load_predictor("models/bert_model")

In [ ]:
#testing again
import time 

message = 'I just broke up with my boyfriend'

start_time = time.time() 
prediction = predictor.predict(message)

print('predicted: {} ({:.2f})'.format(prediction, (time.time() - start_time)))

predicted: sadness (1.23)
